In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Cell: inspect dataset files
import os, glob
base = "/kaggle/input"
print("Top-level folders in /kaggle/input:")
print(os.listdir(base))

# try to find the amazon dataset folder
for d in os.listdir(base):
    if "amazon" in d.lower() or "lokeshparab" in d.lower():
        print("\nPossible dataset folder:", d)
        for f in os.listdir(os.path.join(base, d))[:50]:
            print(" ", f)


Top-level folders in /kaggle/input:
['amazon-products-dataset']

Possible dataset folder: amazon-products-dataset
  Gaming Consoles.csv
  Car Electronics.csv
  Janitorial and Sanitation Supplies.csv
  All Electronics.csv
  All Books.csv
  Make-up.csv
  Travel Accessories.csv
  Indian Language Books.csv
  Car and Bike Care.csv
  Sunglasses.csv
  Bags and Luggage.csv
  Yoga.csv
  Sportswear.csv
  Fiction Books.csv
  Exam Central.csv
  Home Storage.csv
  Toys Gifting Store.csv
  All English.csv
  Amazon-Products.csv
  Air Conditioners.csv
  Shoes.csv
  Casual Shoes.csv
  Baby Products.csv
  Sports Collectibles.csv
  Wallets.csv
  Musical Instruments and Professional Audio.csv
  Gold and Diamond Jewellery.csv
  Nursing and Feeding.csv
  Home Furnishing.csv
  School Textbooks.csv
  All Hindi.csv
  Baby Bath Skin and Grooming.csv
  Coffee Tea and Beverages.csv
  Headphones.csv
  Furniture.csv
  Shirts.csv
  Subscribe and Save.csv
  Fitness Accessories.csv
  Formal Shoes.csv
  Cycling.csv
  W

In [2]:
# ============================================================
# ⚙️ Environment Setup — Hybrid Recommender (Kaggle)
# ============================================================

# Silence noisy dependency warnings
import warnings, os, sys
warnings.filterwarnings("ignore")

print("🐍 Checking Python environment...")
!python3 --version
!pip --version

# ============================================================
# ✅ Install exact compatible versions
# ============================================================
# - scikit-learn pinned below 1.6.0 to satisfy Kaggle preinstalls
# - numpy pinned to 1.26.x (stable)
# - xgboost & imbalanced-learn versions fully compatible with sklearn 1.5.x

!pip install -q \
    scikit-learn==1.5.2 \
    numpy==1.26.4 \
    xgboost==2.0.3 \
    imbalanced-learn==0.12.3 \
    joblib==1.4.2 \
    pandas==2.2.2 \
    scipy==1.13.1

# ============================================================
# ✅ Confirm versions (so you know what’s running)
# ============================================================
import sklearn, numpy, xgboost, imblearn, pandas, scipy, joblib
print(f"scikit-learn: {sklearn.__version__}")
print(f"numpy:        {numpy.__version__}")
print(f"xgboost:      {xgboost.__version__}")
print(f"imblearn:     {imblearn.__version__}")
print(f"pandas:       {pandas.__version__}")
print(f"scipy:        {scipy.__version__}")
print(f"joblib:       {joblib.__version__}")

print("\n✅ Environment ready! You can now safely run the next cells.")


🐍 Checking Python environment...
Python 3.11.13
pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 101.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 104.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 45.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
prep

In [3]:
# Install (Kaggle already has most packages, but safe)
!pip install -q xgboost imbalanced-learn

import warnings
warnings.filterwarnings("ignore")

import os, random, math
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import joblib

random.seed(42)
np.random.seed(42)



In [4]:
# Update this to the folder name you saw in the inspect cell
DATA_FOLDER = "/kaggle/input/amazon-products-dataset"  # <-- change if needed

# list files
files = os.listdir(DATA_FOLDER)
print("Files in dataset folder:", files)

# Try to load best candidate files
# Common patterns: reviews.csv, products.csv, amazon_products.csv
df = None
for fname in files:
    path = os.path.join(DATA_FOLDER, fname)
    if fname.lower().endswith(".csv"):
        print("Trying to read", fname)
        try:
            tmp = pd.read_csv(path, nrows=5)
            print("Sample columns:", tmp.columns.tolist())
            # Heuristic: if it contains 'reviewerID' or 'reviewText' it's reviews
            if any(c.lower() in ['reviewerid','reviewer_id','user_id','user'] for c in tmp.columns):
                # prefer a reviews file
                df = pd.read_csv(path)
                print("Loaded review file:", fname, "shape:", df.shape)
                break
            # else if appears to be products with title/description
            if any(c.lower() in ['title','product_title','name','description'] for c in tmp.columns):
                # keep as product file (if no reviews file found, we will create pseudo interactions)
                if df is None:
                    df = pd.read_csv(path)
                    print("Loaded product file (no reviews found yet):", fname, "shape:", df.shape)
        except Exception as e:
            print("Failed to read", fname, ":", e)

if df is None:
    raise FileNotFoundError("Couldn't auto-detect a CSV file. Please upload or point DATA_FOLDER to the correct dataset.")



Files in dataset folder: ['Gaming Consoles.csv', 'Car Electronics.csv', 'Janitorial and Sanitation Supplies.csv', 'All Electronics.csv', 'All Books.csv', 'Make-up.csv', 'Travel Accessories.csv', 'Indian Language Books.csv', 'Car and Bike Care.csv', 'Sunglasses.csv', 'Bags and Luggage.csv', 'Yoga.csv', 'Sportswear.csv', 'Fiction Books.csv', 'Exam Central.csv', 'Home Storage.csv', 'Toys Gifting Store.csv', 'All English.csv', 'Amazon-Products.csv', 'Air Conditioners.csv', 'Shoes.csv', 'Casual Shoes.csv', 'Baby Products.csv', 'Sports Collectibles.csv', 'Wallets.csv', 'Musical Instruments and Professional Audio.csv', 'Gold and Diamond Jewellery.csv', 'Nursing and Feeding.csv', 'Home Furnishing.csv', 'School Textbooks.csv', 'All Hindi.csv', 'Baby Bath Skin and Grooming.csv', 'Coffee Tea and Beverages.csv', 'Headphones.csv', 'Furniture.csv', 'Shirts.csv', 'Subscribe and Save.csv', 'Fitness Accessories.csv', 'Formal Shoes.csv', 'Cycling.csv', 'Western Wear.csv', 'Bedroom Linen.csv', 'Gaming Ac

In [5]:
# Normalize column names (lowercase)
df.columns = [c.strip() for c in df.columns]

# Heuristics to create interactions + product meta
if any(c.lower() in ['reviewerid','reviewer_id','user'] for c in df.columns):
    # assume review dataset: typical Amazon columns: reviewerID, asin, overall, reviewText, summary, title?
    col_map = {c.lower(): c for c in df.columns}
    # pick columns
    user_col = next((c for c in df.columns if c.lower() in ['reviewerid','reviewer_id','user_id','user']), None)
    item_col = next((c for c in df.columns if c.lower() in ['asin','productid','product_id','product']), None)
    rating_col = next((c for c in df.columns if c.lower() in ['overall','rating','score']), None)
    title_col = next((c for c in df.columns if c.lower() in ['title','product_title','name','headline']), None)
    desc_col = next((c for c in df.columns if c.lower() in ['reviewtext','description','product_description','body']), None)

    print("Detected columns -> user:", user_col, "item:", item_col, "rating:", rating_col, "title:", title_col, "desc:", desc_col)
    interactions = df[[user_col, item_col, rating_col] if rating_col else [user_col, item_col]].copy()
    interactions.columns = ['user_id', 'product_id'] + (['rating'] if rating_col else ['rating'])
    if rating_col is None:
        interactions['rating'] = 1.0
    else:
        interactions['rating'] = interactions['rating'].astype(float)

    # product meta: try to extract from df, else create placeholders
    if title_col is None:
        product_meta = interactions[['product_id']].drop_duplicates().reset_index(drop=True)
        product_meta['title'] = product_meta['product_id'].astype(str)
        product_meta['description'] = ""
        product_meta['category'] = ""
    else:
        product_meta = df[[item_col, title_col]].drop_duplicates().rename(columns={item_col:'product_id', title_col:'title'})
        if desc_col and desc_col in df.columns:
            product_meta = product_meta.merge(df[[item_col, desc_col]].drop_duplicates().rename(columns={item_col:'product_id', desc_col:'description'}), on='product_id', how='left')
        else:
            product_meta['description'] = ""
        if 'category' not in product_meta.columns:
            product_meta['category'] = ""
else:
    # product-only file: create synthetic interactions (lightweight)
    print("Dataset appears to be product metadata only. Creating synthetic demo interactions.")
    product_meta = df.copy()
    if 'product_id' not in product_meta.columns:
        # choose a column to be product id (first column)
        product_meta = product_meta.rename(columns={product_meta.columns[0]:'product_id'})
    if 'title' not in product_meta.columns:
        product_meta['title'] = product_meta['product_id'].astype(str)
    if 'description' not in product_meta.columns:
        product_meta['description'] = product_meta['title'].astype(str)
    # synthesize interactions: random users picking products
    n_users = 1000
    interactions = []
    products = product_meta['product_id'].unique().tolist()
    for u in range(n_users):
        picked = random.sample(products, k=min(20, len(products)))
        for p in picked:
            interactions.append({'user_id': f"U{u}", 'product_id': p, 'rating': 1.0})
    interactions = pd.DataFrame(interactions)

# quick shapes and samples
print("Interactions shape:", interactions.shape)
print("Product meta shape:", product_meta.shape)
interactions.head(), product_meta.head()


Dataset appears to be product metadata only. Creating synthetic demo interactions.
Interactions shape: (0, 0)
Product meta shape: (0, 11)


(Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: [product_id, main_category, sub_category, image, link, ratings, no_of_ratings, discount_price, actual_price, title, description]
 Index: [])

In [6]:
# ============================================================
# 🧠 FIXED: Auto-load Amazon dataset and create interactions
# Works for "lokeshparab/amazon-products-dataset" and similar
# ============================================================

import pandas as pd, os, random

# Point to your Kaggle dataset folder
DATA_FOLDER = "/kaggle/input/amazon-products-dataset"  # ✅ adjust if name differs
print("Files in dataset folder:", os.listdir(DATA_FOLDER))

# Try to read any CSV file in that folder
df = None
for fname in os.listdir(DATA_FOLDER):
    if fname.lower().endswith(".csv"):
        path = os.path.join(DATA_FOLDER, fname)
        try:
            tmp = pd.read_csv(path, nrows=5)
            print(f"✅ Loaded sample of {fname}")
            print(tmp.columns.tolist())
            df = pd.read_csv(path)
            break
        except Exception as e:
            print(f"⚠️ Could not read {fname}: {e}")

if df is None:
    raise ValueError("No readable CSV file found in dataset folder.")

print("Full dataset shape:", df.shape)

# ---------------------------------------------
# 🔧 Step 2 — Auto-detect and rename columns
# ---------------------------------------------
rename_map = {}

for c in df.columns:
    cl = c.lower()
    if cl in ['reviewerid','reviewer_id','user','userid','user_id','customerid','customer_id']:
        rename_map[c] = 'user_id'
    if cl in ['asin','product','productid','product_id','item','itemid','item_id','sku']:
        rename_map[c] = 'product_id'
    if cl in ['rating','overall','score','stars']:
        rename_map[c] = 'rating'
    if cl in ['title','product_title','name','summary','headline']:
        rename_map[c] = 'title'
    if cl in ['reviewtext','description','product_description','body']:
        rename_map[c] = 'description'
    if 'category' in cl:
        rename_map[c] = 'category'

df = df.rename(columns=rename_map)
print("Renamed columns:", rename_map)
print("Columns after renaming:", df.columns.tolist())

# ---------------------------------------------
# 🔧 Step 3 — Build interactions + product meta
# ---------------------------------------------
# If dataset has users & products -> use real interactions
if 'user_id' in df.columns and 'product_id' in df.columns:
    print("✅ Detected review-style dataset (real interactions).")
    interactions = df[['user_id', 'product_id']].copy()
    if 'rating' in df.columns:
        interactions['rating'] = df['rating'].astype(float)
    else:
        interactions['rating'] = 1.0

    # product meta
    meta_cols = ['product_id']
    for c in ['title','description','category']:
        if c in df.columns:
            meta_cols.append(c)
    product_meta = df[meta_cols].drop_duplicates('product_id').reset_index(drop=True)

# If only product info exists -> create synthetic interactions
else:
    print("⚠️ No user_id or product_id columns found — creating synthetic demo interactions.")
    if 'product_id' not in df.columns:
        df = df.rename(columns={df.columns[0]: 'product_id'})
    if 'title' not in df.columns:
        df['title'] = df['product_id'].astype(str)
    if 'description' not in df.columns:
        df['description'] = df['title'].astype(str)
    product_meta = df[['product_id','title','description']].drop_duplicates('product_id')
    
    n_users = 500
    interactions = []
    products = product_meta['product_id'].tolist()
    for u in range(n_users):
        picked = random.sample(products, k=min(15, len(products)))
        for p in picked:
            interactions.append({'user_id': f'U{u}', 'product_id': p, 'rating': 1.0})
    interactions = pd.DataFrame(interactions)

print("\n✅ interactions shape:", interactions.shape)
print("✅ product_meta shape:", product_meta.shape)
print("\nSample of interactions:")
display(interactions.head())


Files in dataset folder: ['Gaming Consoles.csv', 'Car Electronics.csv', 'Janitorial and Sanitation Supplies.csv', 'All Electronics.csv', 'All Books.csv', 'Make-up.csv', 'Travel Accessories.csv', 'Indian Language Books.csv', 'Car and Bike Care.csv', 'Sunglasses.csv', 'Bags and Luggage.csv', 'Yoga.csv', 'Sportswear.csv', 'Fiction Books.csv', 'Exam Central.csv', 'Home Storage.csv', 'Toys Gifting Store.csv', 'All English.csv', 'Amazon-Products.csv', 'Air Conditioners.csv', 'Shoes.csv', 'Casual Shoes.csv', 'Baby Products.csv', 'Sports Collectibles.csv', 'Wallets.csv', 'Musical Instruments and Professional Audio.csv', 'Gold and Diamond Jewellery.csv', 'Nursing and Feeding.csv', 'Home Furnishing.csv', 'School Textbooks.csv', 'All Hindi.csv', 'Baby Bath Skin and Grooming.csv', 'Coffee Tea and Beverages.csv', 'Headphones.csv', 'Furniture.csv', 'Shirts.csv', 'Subscribe and Save.csv', 'Fitness Accessories.csv', 'Formal Shoes.csv', 'Cycling.csv', 'Western Wear.csv', 'Bedroom Linen.csv', 'Gaming Ac

""


In [7]:
# Robust loader: pick a good CSV, create product_meta + interactions (falls back to synthetic)
import os, pandas as pd, random
from scipy.sparse import csr_matrix

DATA_FOLDER = "/kaggle/input/amazon-products-dataset"  # adjust if needed

# 1) Inspect CSV files and sizes / row counts
csv_files = [f for f in os.listdir(DATA_FOLDER) if f.lower().endswith('.csv')]
info = []
print("Found CSV files (name, bytes, rows sample):\n")
for f in csv_files:
    path = os.path.join(DATA_FOLDER, f)
    try:
        size = os.path.getsize(path)
        # quick check rows (read small nrows to avoid heavy io)
        tmp = pd.read_csv(path, nrows=5)
        rows_sample = len(tmp)
        info.append((f, size, rows_sample))
        print(f" - {f:60s}  {size:10d} bytes  sample_rows={rows_sample}  cols={list(tmp.columns)[:6]}")
    except Exception as e:
        print(f" - {f:60s}  FAILED to read: {e}")

# 2) choose the largest non-empty CSV by bytes (skips very small files)
candidates = [t for t in info if t[1] > 200 and t[2] > 0]  # threshold: >200 bytes and sample rows >0
if not candidates:
    # relax threshold if nothing found
    candidates = [t for t in info if t[2] > 0]
if not candidates:
    raise FileNotFoundError("No non-empty CSV files found in dataset folder. Pick a different dataset or upload files.")

# Sort candidates by file size descending and pick top
candidates.sort(key=lambda x: x[1], reverse=True)
chosen_fname = candidates[0][0]
print(f"\n✅ Choosing file: {chosen_fname}\n")
chosen_path = os.path.join(DATA_FOLDER, chosen_fname)

# 3) load the chosen CSV (full read)
df = pd.read_csv(chosen_path)
print("Loaded shape:", df.shape)
print("Columns (sample):", df.columns.tolist()[:30])

# 4) standardize and detect columns
rename_map = {}
for c in df.columns:
    cl = c.strip().lower()
    if cl in ['reviewerid','reviewer_id','user','userid','user_id','customerid','customer_id']:
        rename_map[c] = 'user_id'
    if cl in ['asin','product','productid','product_id','item','itemid','item_id','sku','id','productId'.lower()]:
        # prefer asin/product-like names
        rename_map[c] = 'product_id'
    if cl in ['rating','overall','score','stars']:
        rename_map[c] = 'rating'
    if cl in ['title','product_title','name','summary','headline']:
        rename_map[c] = 'title'
    if cl in ['reviewtext','description','product_description','body','desc']:
        rename_map[c] = 'description'
    if 'category' in cl:
        # if there are multiple category columns we'll keep both temporarily
        rename_map[c] = 'category'

df = df.rename(columns=rename_map)
print("After renaming, columns include:", list(df.columns)[:30])

# 5) create product_meta
# Try to gather product_id/title/description/category. If not present, synthesize product_id.
product_meta = None
if 'product_id' in df.columns:
    # Try to collect title/description/category if present
    meta_cols = ['product_id']
    for c in ['title','description','category']:
        if c in df.columns:
            meta_cols.append(c)
    # drop duplicates on product_id
    product_meta = df[meta_cols].drop_duplicates('product_id').reset_index(drop=True)
else:
    # No product id column; try to construct one from 'link' or index
    if 'link' in df.columns:
        df['product_id'] = df['link'].astype(str)
    else:
        df['product_id'] = df.index.astype(str).map(lambda x: f"P{x}")
    # ensure title and description exist
    if 'title' not in df.columns:
        df['title'] = df.iloc[:,0].astype(str)
    if 'description' not in df.columns:
        df['description'] = df['title'].astype(str)
    product_meta = df[['product_id','title','description']].drop_duplicates('product_id').reset_index(drop=True)

print("product_meta sample shape:", product_meta.shape)
display(product_meta.head())

# 6) Create interactions if user info present; otherwise synthesize interactions
if 'user_id' in df.columns and 'product_id' in df.columns:
    print("Detected user interactions in file. Building interactions from real data.")
    interactions = df[['user_id','product_id']].copy()
    interactions['rating'] = df['rating'].astype(float) if 'rating' in df.columns else 1.0
else:
    # synthesize interactions — ensure we have some products to sample
    n_products = product_meta.shape[0]
    if n_products == 0:
        raise RuntimeError("No products found to create interactions from. Try a different file in the dataset.")
    print("No user info detected. Creating synthetic interactions.")
    n_users = 500
    interactions_list = []
    all_products = product_meta['product_id'].tolist()
    # if dataset is tiny, reduce users; ensure each user has min 5 interactions
    interactions_per_user = min(25, max(5, n_products//10))
    for u in range(n_users):
        chosen = random.sample(all_products, k=min(interactions_per_user, len(all_products)))
        for p in chosen:
            interactions_list.append({'user_id': f"U{u}", 'product_id': p, 'rating': 1.0})
    interactions = pd.DataFrame(interactions_list)

print("interactions shape before pruning:", interactions.shape)
display(interactions.head())

# 7) Prune rare users/items (optional)
min_user_interactions = 3
min_item_interactions = 3
user_counts = interactions['user_id'].value_counts()
valid_users = user_counts[user_counts >= min_user_interactions].index
item_counts = interactions['product_id'].value_counts()
valid_items = item_counts[item_counts >= min_item_interactions].index

interactions = interactions[interactions['user_id'].isin(valid_users) & interactions['product_id'].isin(valid_items)].copy()
print("interactions shape after pruning:", interactions.shape)

# 8) Ensure product_meta only contains products present in interactions
product_meta = product_meta[product_meta['product_id'].isin(interactions['product_id'].unique())].reset_index(drop=True)
print("product_meta shape after filtering:", product_meta.shape)

# 9) Build mappings and sparse matrix
user_ids = interactions['user_id'].unique().tolist()
item_ids = interactions['product_id'].unique().tolist()
user_to_index = {u:i for i,u in enumerate(user_ids)}
index_to_user = {i:u for u,i in user_to_index.items()}
item_to_index = {p:i for i,p in enumerate(item_ids)}
index_to_item = {i:p for p,i in item_to_index.items()}

rows = interactions['user_id'].map(user_to_index)
cols = interactions['product_id'].map(item_to_index)
data = interactions['rating'].astype(float)
user_item_csr = csr_matrix((data, (rows, cols)), shape=(len(user_ids), len(item_ids)))

print("\n✅ Final diagnostics:")
print(" - chosen CSV:", chosen_fname)
print(" - interactions (users x items):", user_item_csr.shape)
print(" - unique users:", len(user_ids), "unique items:", len(item_ids))
print(" - sample users:", user_ids[:3])
print(" - sample items:", item_ids[:3])


Found CSV files (name, bytes, rows sample):

 - Gaming Consoles.csv                                                   94 bytes  sample_rows=0  cols=['name', 'main_category', 'sub_category', 'image', 'link', 'ratings']
 - Car Electronics.csv                                               378730 bytes  sample_rows=5  cols=['name', 'main_category', 'sub_category', 'image', 'link', 'ratings']
 - Janitorial and Sanitation Supplies.csv                            355062 bytes  sample_rows=5  cols=['name', 'main_category', 'sub_category', 'image', 'link', 'ratings']
 - All Electronics.csv                                              3691715 bytes  sample_rows=5  cols=['name', 'main_category', 'sub_category', 'image', 'link', 'ratings']
 - All Books.csv                                                         94 bytes  sample_rows=0  cols=['name', 'main_category', 'sub_category', 'image', 'link', 'ratings']
 - Make-up.csv                                                       891819 bytes  sample_

,product_id,title,description
0,https://www.amazon.in/Lloyd-Inverter-Convertib...,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...
1,https://www.amazon.in/LG-Convertible-Anti-Viru...,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...
2,https://www.amazon.in/LG-Inverter-Convertible-...,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...
3,https://www.amazon.in/LG-Convertible-Anti-Viru...,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...
4,https://www.amazon.in/Carrier-Inverter-Split-C...,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...


No user info detected. Creating synthetic interactions.
interactions shape before pruning: (12500, 3)


,user_id,product_id,rating
0,U0,https://www.amazon.in/ORENAME-Invisible-Silico...,1.0
1,U0,https://www.amazon.in/Shrey-SHREY-Rucksack-Bag...,1.0
2,U0,https://www.amazon.in/MERSODA%C2%AE-Stylish-Co...,1.0
3,U0,https://www.amazon.in/AmazonBasics-Lightning-C...,1.0
4,U0,https://www.amazon.in/Zebronics-Zeb-Symphony-B...,1.0


interactions shape after pruning: (0, 3)
product_meta shape after filtering: (0, 3)

✅ Final diagnostics:
 - chosen CSV: Amazon-Products.csv
 - interactions (users x items): (0, 0)
 - unique users: 0 unique items: 0
 - sample users: []
 - sample items: []


In [8]:
# ============================================================
# 🧠 Train/Test Split (per-user holdout)
# ============================================================

import random
random.seed(42)

def train_test_holdout(df, user_col='user_id', item_col='product_id'):
    """Splits data so that each user has 1 item held out for testing."""
    train_idx, test_idx = [], []
    for user, group in df.groupby(user_col):
        idxs = group.index.tolist()
        if len(idxs) <= 1:
            train_idx.extend(idxs)
        else:
            holdout = random.choice(idxs)
            test_idx.append(holdout)
            train_idx.extend([i for i in idxs if i != holdout])
    train_df = df.loc[train_idx].reset_index(drop=True)
    test_df = df.loc[test_idx].reset_index(drop=True)
    return train_df, test_df

train_df, test_df = train_test_holdout(interactions)

print("Train interactions:", train_df.shape[0])
print("Test interactions:", test_df.shape[0])

# Rebuild sparse train matrix
from scipy.sparse import csr_matrix
rows = train_df['user_id'].map(user_to_index)
cols = train_df['product_id'].map(item_to_index)
data = train_df['rating'].astype(float)
train_csr = csr_matrix((data, (rows, cols)), shape=(len(user_ids), len(item_ids)))
print("Train CSR shape:", train_csr.shape)


Train interactions: 0
Test interactions: 0
Train CSR shape: (0, 0)


In [14]:
# Single cell: load products CSV -> build product_meta -> TF-IDF -> SVD -> Annoy neighbors
# Paste and run in Kaggle/Jupyter.

# If annoy is not installed in your kernel, uncomment:
# !pip install annoy

import os, re, time, joblib, math
import numpy as np, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from annoy import AnnoyIndex

start = time.time()
print("Start pipeline...")

# 1) Find a likely Amazon products CSV under /kaggle/input
def find_product_csv(input_root="/kaggle/input"):
    if not os.path.exists(input_root):
        return None
    for root, dirs, files in os.walk(input_root):
        for f in files:
            fn = f.lower()
            if fn.endswith(".csv") and ("amazon" in fn or "product" in fn or "products" in fn):
                # prefer files with 'amazon' and 'product' in name
                return os.path.join(root, f)
    return None

csv_path = find_product_csv()
if csv_path is None:
    raise FileNotFoundError("No product CSV found under /kaggle/input. Upload your dataset or set `csv_path` manually.")

print("Loading CSV:", csv_path)
# read with robust settings
df = pd.read_csv(csv_path, dtype=str, encoding='utf-8', engine='c')
print("Raw dataframe shape:", df.shape)
display(df.head(3))

# 2) Build product_meta: pick sensible columns and ensure product_id + title exist
df_columns = [c.strip() for c in df.columns]
df.columns = df_columns

# attempt to find ID column
id_candidates = [c for c in df.columns if c.lower() in ('product_id','id','asin','sku')]
prod_id_col = id_candidates[0] if id_candidates else None
if prod_id_col is None:
    # create synthetic id
    df.insert(0, 'product_id', [f"P{i}" for i in range(len(df))])
    prod_id_col = 'product_id'

# attempt to find title/name column
title_candidates = [c for c in df.columns if c.lower() in ('title','name','product_name','product title')]
title_col = title_candidates[0] if title_candidates else None
if title_col is None:
    # fallback to first object-type column that is not url/image
    obj_cols = [c for c in df.columns if df[c].dtype == 'object']
    def looks_like_url(col):
        sample = df[col].dropna().astype(str).head(8).tolist()
        if not sample: return False
        return sum(1 for s in sample if re.search(r'https?://|www\.|\.jpg|\.png|/dp/|amazon\.', s.lower())) >= 1
    obj_cols = [c for c in obj_cols if not looks_like_url(c)]
    title_col = obj_cols[0] if obj_cols else prod_id_col

# create product_meta with columns: product_id, title plus everything else kept
product_meta = df.copy()
product_meta['product_id'] = product_meta[prod_id_col].astype(str)
product_meta['title'] = product_meta[title_col].astype(str) if title_col in product_meta.columns else product_meta['product_id'].astype(str)

# 3) Build a robust text field from available textual columns
preferred_text_cols = ['title','name','product_name','description','desc','category','categories','brand','features','specs']
available = [c for c in preferred_text_cols if c in product_meta.columns]
# also add other object dtype columns (excluding urls/images)
if not available:
    obj_cols = [c for c in product_meta.columns if product_meta[c].dtype == 'object']
    def is_url_col(col):
        sample = product_meta[col].dropna().astype(str).head(6).tolist()
        if not sample: return False
        return sum(1 for s in sample if re.search(r'https?://|www\.|\.jpg|\.png|/dp/|amazon\.', s.lower())) >= 1
    obj_cols = [c for c in obj_cols if not is_url_col(c)]
    available = obj_cols[:3]  # pick a few

if 'title' not in available:
    available = ['title'] + available

print("Text source columns used:", available)

def build_text(row):
    parts = []
    for c in available:
        if c in row and pd.notna(row[c]):
            s = str(row[c]).strip()
            if s and not re.match(r'^\s*https?://', s):
                parts.append(s)
    if parts:
        return " ".join(parts)
    # fallback to product_id
    if pd.notna(row['product_id']):
        return str(row['product_id'])
    return ""

product_meta['text'] = product_meta.apply(build_text, axis=1)

# 4) Coerce to safe strings (handle lists/dicts), drop empties
def to_safe_text(x):
    if x is None:
        return ""
    if isinstance(x, (list, tuple, set)):
        return " ".join(str(i) for i in x if i is not None)
    if isinstance(x, dict):
        return " ".join(f"{k} {v}" for k,v in x.items())
    return str(x)
product_meta['text'] = product_meta['text'].apply(to_safe_text).map(lambda s: s.strip())
product_meta.loc[product_meta['text'] == "", 'text'] = pd.NA

print("Non-empty text rows (before fallback):", int(product_meta['text'].notna().sum()), "/", len(product_meta))

# If too few rows have text, create fallback tokens from title/product_id
min_usable = max(1, int(0.01 * len(product_meta)))  # require at least 1% usable docs
if int(product_meta['text'].notna().sum()) < min_usable:
    print("Too few usable text rows; creating fallback text from title/product_id...")
    def fallback_text(row):
        t = row.get('title') or ""
        if pd.notna(t) and str(t).strip():
            toks = re.findall(r'\b[a-zA-Z0-9]{2,}\b', str(t))
            return " ".join(toks[:6]) if toks else ("product_" + str(row.name))
        if pd.notna(row.get('product_id')):
            return str(row.get('product_id'))
        return "product_" + str(row.name)
    mask = product_meta['text'].isna()
    product_meta.loc[mask, 'text'] = product_meta.loc[mask].apply(fallback_text, axis=1)

product_meta = product_meta.dropna(subset=['text']).reset_index(drop=True)
print("Final product_meta shape:", product_meta.shape)
display(product_meta[['product_id','title','text']].head(5))

# 5) Clean text for TF-IDF
def clean_text(s):
    s = str(s).lower()
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

product_meta['text_clean'] = product_meta['text'].astype(str).map(clean_text)
product_meta['text_len'] = product_meta['text_clean'].str.len().fillna(0).astype(int)
print("Average cleaned text length:", product_meta['text_len'].mean())

# 6) TF-IDF (sparse)
max_features = 5000
print("Computing TF-IDF (max_features=", max_features, ") ...", sep="")
tfidf = TfidfVectorizer(max_features=max_features, stop_words='english', token_pattern=r'(?u)\b[A-Za-z0-9_]{2,}\b', min_df=1)
tfidf_matrix = tfidf.fit_transform(product_meta['text_clean'].astype(str).values)
print("TF-IDF shape:", tfidf_matrix.shape)

# 7) Use SVD to get dense embeddings (lower-dim) for ANN
n_items = tfidf_matrix.shape[0]
if n_items <= 0:
    raise RuntimeError("No items to index. Check product_meta.")
# choose components
if n_items > 20000:
    n_comp = 128
elif n_items > 5000:
    n_comp = 96
else:
    n_comp = min(64, max(8, n_items//10))
print("Using TruncatedSVD with n_components =", n_comp)
svd = TruncatedSVD(n_components=n_comp, random_state=42)
item_embeds = svd.fit_transform(tfidf_matrix)   # dense (n_items x n_comp)
item_embeds = normalize(item_embeds, axis=1)
print("Embeddings shape:", item_embeds.shape)

# 8) Build Annoy index (angular ~ cosine)
annoy_dim = item_embeds.shape[1]
ann_index = AnnoyIndex(annoy_dim, metric='angular')
n_trees = 50 if n_items > 5000 else 20
print(f"Building Annoy index with {n_trees} trees (this may take a moment)...")
for i in range(n_items):
    ann_index.add_item(i, item_embeds[i].astype('float32'))
ann_index.build(n_trees)
print("Annoy index built.")

# 9) Precompute top-K neighbors per item (fast lookups later)
top_k = 16
similarity_map = {}
for i in range(n_items):
    # get neighbors (self included as first result)
    nn = ann_index.get_nns_by_item(i, top_k+1, include_distances=False)
    # exclude self
    nn = [j for j in nn if j != i][:top_k]
    similarity_map[product_meta.at[i,'product_id']] = [product_meta.at[j,'product_id'] for j in nn]
# show sample
sample_items = list(similarity_map.keys())[:3]
print("Sample similarity_map entries (first 3):")
for s in sample_items:
    print(" ", s, "->", similarity_map[s][:6])

# 10) Save artifacts
joblib.dump(tfidf, "tfidf_vectorizer.joblib")
joblib.dump({'svd': svd}, "svd_model.joblib")
joblib.dump(product_meta[['product_id','title','text_clean']].reset_index(drop=True), "product_meta.joblib")
ann_index.save("annoy_index.ann")
joblib.dump(similarity_map, "similarity_map.joblib")
print("Saved artifacts: tfidf_vectorizer.joblib, svd_model.joblib, product_meta.joblib, annoy_index.ann, similarity_map.joblib")

elapsed = time.time() - start
print(f"Done in {elapsed:.1f}s. Indexed {n_items} products.")


Start pipeline...
Loading CSV: /kaggle/input/amazon-products-dataset/Amazon-Products.csv
Raw dataframe shape: (551585, 10)


,Unnamed: 0,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990"
1,1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990"
2,2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990"


Text source columns used: ['title', 'name']
Non-empty text rows (before fallback): 551585 / 551585
Final product_meta shape: (551585, 13)


,product_id,title,text
0,P0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...
1,P1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...
2,P2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...
3,P3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...
4,P4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...


Average cleaned text length: 154.20481158842244
Computing TF-IDF (max_features=5000) ...
TF-IDF shape: (551585, 5000)
Using TruncatedSVD with n_components = 128
Embeddings shape: (551585, 128)
Building Annoy index with 50 trees (this may take a moment)...
Annoy index built.
Sample similarity_map entries (first 3):
  P0 -> ['P6', 'P7', 'P16', 'P59', 'P69', 'P747']
  P1 -> ['P3', 'P22', 'P35', 'P233', 'P804', 'P834']
  P2 -> ['P274', 'P810', 'P243802', 'P248969', 'P339937', 'P101']
Saved artifacts: tfidf_vectorizer.joblib, svd_model.joblib, product_meta.joblib, annoy_index.ann, similarity_map.joblib
Done in 222.8s. Indexed 551585 products.


In [15]:
# ============================================================
# Robust loader: choose a large CSV, build product_meta + interactions
# - merges multiple category columns safely
# - creates synthetic users/interactions if no user data present
# - builds user_to_index, item_to_index, and train_csr (sparse matrix)
# ============================================================

import os, random, pandas as pd, numpy as np
from scipy.sparse import csr_matrix
random.seed(42)
np.random.seed(42)

# ======= ADJUST IF YOUR DATASET PATH IS DIFFERENT =======
DATA_FOLDER = "/kaggle/input/amazon-products-dataset"
# ========================================================

# 1) List CSV files & sizes, prefer largest non-empty CSV
csv_files = [f for f in os.listdir(DATA_FOLDER) if f.lower().endswith('.csv')]
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {DATA_FOLDER}")

file_info = []
for f in csv_files:
    path = os.path.join(DATA_FOLDER, f)
    try:
        size = os.path.getsize(path)
        # quick peek to ensure not empty & to show headers
        sample = pd.read_csv(path, nrows=5)
        file_info.append((f, size, sample.shape[0], list(sample.columns)))
    except Exception as e:
        # skip unreadable files
        print(f"Skipping {f}: {e}")

# Sort by file size descending
file_info.sort(key=lambda x: x[1], reverse=True)
print("Top CSV candidates (name, size_bytes, sample_rows, sample_cols):")
for info in file_info[:10]:
    print(" -", info)

# Pick the largest file that has at least 20 rows in full read, else pick largest available
chosen_fname = None
for f, size, sample_rows, cols in file_info:
    if size > 1024 and sample_rows > 0:
        # try full read to check row count (safe; dataset sizes may be large but Kaggle can handle)
        path = os.path.join(DATA_FOLDER, f)
        try:
            df_tmp = pd.read_csv(path)
            if df_tmp.shape[0] >= 20:
                chosen_fname = f
                df = df_tmp
                break
        except Exception:
            continue

# fallback: choose biggest readable file
if chosen_fname is None:
    if not file_info:
        raise FileNotFoundError("No readable CSV files in dataset folder.")
    chosen_fname = file_info[0][0]
    df = pd.read_csv(os.path.join(DATA_FOLDER, chosen_fname))

print(f"\n✅ Chosen file: {chosen_fname} — shape {df.shape}")
print("Sample columns:", list(df.columns)[:50])

# 2) Intelligent renaming of common columns
rename_map = {}
for c in df.columns:
    cl = c.strip().lower()
    if cl in ['reviewerid','reviewer_id','user','userid','user_id','customerid','customer_id']:
        rename_map[c] = 'user_id'
    if cl in ['asin','product','productid','product_id','item','itemid','item_id','sku','id']:
        # heuristic: prefer ASIN-like or generic id names
        rename_map[c] = 'product_id'
    if cl in ['rating','overall','score','stars']:
        rename_map[c] = 'rating'
    if cl in ['title','product_title','name','summary','headline']:
        rename_map[c] = 'title'
    if cl in ['reviewtext','description','product_description','body','desc']:
        rename_map[c] = 'description'
    if 'category' in cl:
        # capture main_category, sub_category, etc.
        rename_map[c] = 'category'

# apply rename map (only keys present)
df = df.rename(columns=rename_map)
print("Applied renames (if any):", rename_map)
print("Columns after rename (sample):", list(df.columns)[:50])

# 3) Build product_meta safely (join multiple category columns if present)
# If there are multiple original category-like columns they will have been renamed to 'category' possibly multiple times;
# find all original column names that contained 'category' then join them.
cat_cols = [orig for orig in df.columns if 'category' in orig.lower()]
# But since we normalized into 'category' above, also check for duplicate columns by original names:
# Build list of columns from original df (before rename) that contained 'category' is already found above as cat_cols.
# Use the existing df columns and attempt to gather all category-like columns (heuristic: columns with 'category' or 'cat' in name)
cat_candidates = [c for c in df.columns if 'category' in c.lower() or c.lower().endswith('cat') or c.lower().endswith('_cat')]
cat_cols = list(dict.fromkeys(cat_cols + cat_candidates))  # unique preserve order

# Ensure product_id, title, description exist (synthesize if not)
if 'product_id' not in df.columns:
    # try to derive from 'link' or index
    if 'link' in df.columns:
        df['product_id'] = df['link'].astype(str)
    else:
        df['product_id'] = df.index.astype(str).map(lambda x: f"P{x}")

if 'title' not in df.columns:
    # best-effort: use first string-like column
    for c in df.columns:
        if df[c].dtype == object and c != 'product_id':
            df['title'] = df[c].astype(str)
            break
    if 'title' not in df.columns:
        df['title'] = df['product_id'].astype(str)

if 'description' not in df.columns:
    df['description'] = df['title'].astype(str)

# Build product_meta with a single 'category' column that aggregates category-like columns
if cat_cols:
    # convert all cat_cols to string and join them into one text column
    product_meta = df[['product_id', 'title', 'description']].copy()
    # some cat_cols may not actually be in df due to prior renames; filter
    cat_cols_present = [c for c in cat_cols if c in df.columns]
    if cat_cols_present:
        product_meta['category'] = df[cat_cols_present].astype(str).agg(' '.join, axis=1)
    else:
        product_meta['category'] = "Unknown"
else:
    product_meta = df[['product_id', 'title', 'description']].copy()
    product_meta['category'] = "Unknown"

product_meta = product_meta.drop_duplicates('product_id').reset_index(drop=True)

print(f"\n✅ product_meta built: {product_meta.shape[0]} unique products")
display(product_meta.head())

# 4) Build interactions
if 'user_id' in df.columns and 'product_id' in df.columns:
    print("Detected user interactions in the file -> building interactions from real data.")
    interactions = df[['user_id', 'product_id']].copy()
    if 'rating' in df.columns:
        interactions['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(1.0)
    else:
        interactions['rating'] = 1.0
else:
    # No user info -> generate synthetic users with random interactions
    print("No user info found. Generating synthetic users & interactions.")
    n_products = product_meta.shape[0]
    if n_products == 0:
        raise RuntimeError("No products available in chosen CSV to generate interactions. Pick a different file.")
    n_users = 800
    interactions_list = []
    all_products = product_meta['product_id'].tolist()
    for u in range(n_users):
        # each synthetic user interacts with 10-25 random products
        k = min(len(all_products), random.randint(10, min(25, max(10, len(all_products)//10))))
        chosen = random.sample(all_products, k=k)
        for pid in chosen:
            interactions_list.append({'user_id': f"U{u}", 'product_id': pid, 'rating': 1.0})
    interactions = pd.DataFrame(interactions_list)

print(f"\n✅ interactions built (before pruning): {interactions.shape[0]} rows")
display(interactions.head())

# 5) Light pruning to remove extremely rare users/items (set thresholds to 1 to be permissive)
min_user_interactions = 1
min_item_interactions = 1

user_counts = interactions['user_id'].value_counts()
valid_users = user_counts[user_counts >= min_user_interactions].index
item_counts = interactions['product_id'].value_counts()
valid_items = item_counts[item_counts >= min_item_interactions].index

interactions = interactions[
    interactions['user_id'].isin(valid_users) & interactions['product_id'].isin(valid_items)
].copy()

print("After pruning interactions:", interactions.shape)

# 6) Filter product_meta to only products present in interactions
product_meta = product_meta[product_meta['product_id'].isin(interactions['product_id'].unique())].reset_index(drop=True)
print("Filtered product_meta shape:", product_meta.shape)

# 7) Build mappings and sparse matrix (train_csr)
user_ids = interactions['user_id'].unique().tolist()
item_ids = interactions['product_id'].unique().tolist()
user_to_index = {u:i for i,u in enumerate(user_ids)}
index_to_user = {i:u for u,i in user_to_index.items()}
item_to_index = {p:i for i,p in enumerate(item_ids)}
index_to_item = {i:p for p,i in item_to_index.items()}

rows = interactions['user_id'].map(user_to_index)
cols = interactions['product_id'].map(item_to_index)
data = interactions['rating'].astype(float)

train_csr = csr_matrix((data, (rows, cols)), shape=(len(user_ids), len(item_ids)))

print("\n✅ Final diagnostics:")
print(" - chosen CSV:", chosen_fname)
print(" - interactions rows:", interactions.shape[0])
print(" - unique users:", len(user_ids))
print(" - unique items:", len(item_ids))
print(" - train_csr shape (users x items):", train_csr.shape)

# variables available for next steps: interactions, product_meta, user_to_index, item_to_index, train_csr


Top CSV candidates (name, size_bytes, sample_rows, sample_cols):
 - ('Amazon-Products.csv', 188603963, 5, ['Unnamed: 0', 'name', 'main_category', 'sub_category', 'image', 'link', 'ratings', 'no_of_ratings', 'discount_price', 'actual_price'])
 - ('Fashion and Silver Jewellery.csv', 7045316, 5, ['name', 'main_category', 'sub_category', 'image', 'link', 'ratings', 'no_of_ratings', 'discount_price', 'actual_price'])
 - ('Jewellery.csv', 6730217, 5, ['name', 'main_category', 'sub_category', 'image', 'link', 'ratings', 'no_of_ratings', 'discount_price', 'actual_price'])
 - ('Lingerie and Nightwear.csv', 6609118, 5, ['name', 'main_category', 'sub_category', 'image', 'link', 'ratings', 'no_of_ratings', 'discount_price', 'actual_price'])
 - ('Bags and Luggage.csv', 6563675, 5, ['name', 'main_category', 'sub_category', 'image', 'link', 'ratings', 'no_of_ratings', 'discount_price', 'actual_price'])
 - ('Clothing.csv', 6402549, 5, ['name', 'main_category', 'sub_category', 'image', 'link', 'ratings

,product_id,title,description,category
0,https://www.amazon.in/Lloyd-Inverter-Convertib...,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances Air Conditioners
1,https://www.amazon.in/LG-Convertible-Anti-Viru...,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances Air Conditioners
2,https://www.amazon.in/LG-Inverter-Convertible-...,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances Air Conditioners
3,https://www.amazon.in/LG-Convertible-Anti-Viru...,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances Air Conditioners
4,https://www.amazon.in/Carrier-Inverter-Split-C...,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances Air Conditioners


No user info found. Generating synthetic users & interactions.

✅ interactions built (before pruning): 14046 rows


,user_id,product_id,rating
0,U0,https://www.amazon.in/Shrey-SHREY-Rucksack-Bag...,1.0
1,U0,https://www.amazon.in/MERSODA%C2%AE-Stylish-Co...,1.0
2,U0,https://www.amazon.in/AmazonBasics-Lightning-C...,1.0
3,U0,https://www.amazon.in/Zebronics-Zeb-Symphony-B...,1.0
4,U0,https://www.amazon.in/BIBA-Womens-Anarkali-SKD...,1.0


After pruning interactions: (14046, 3)
Filtered product_meta shape: (13873, 4)

✅ Final diagnostics:
 - chosen CSV: Amazon-Products.csv
 - interactions rows: 14046
 - unique users: 800
 - unique items: 13873
 - train_csr shape (users x items): (800, 13873)


In [16]:
# ============================================================
# 🧠 Content-based encoding (TF-IDF)
# ============================================================
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Combine title + description + category into one text column
def combine_text(row):
    return " ".join(str(x) for x in [row.get('title',''), row.get('description',''), row.get('category','')] if pd.notna(x))

product_meta['text'] = product_meta.apply(combine_text, axis=1)

tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
tfidf_matrix = tfidf.fit_transform(product_meta['text'].fillna(''))
prod_cosine = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("✅ TF-IDF matrix shape:", tfidf_matrix.shape)
print("✅ Cosine similarity shape:", prod_cosine.shape)


✅ TF-IDF matrix shape: (13873, 5000)
✅ Cosine similarity shape: (13873, 13873)


In [17]:
# ============================================================
# 🧩 Collaborative filtering with Truncated SVD
# ============================================================
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

n_users, n_items = train_csr.shape
if n_items < 2:
    raise ValueError(f"❌ Too few unique products ({n_items}). Pick a larger CSV or lower pruning.")

# choose components safely
n_components = min(50, max(2, min(n_users, n_items) - 1))
print(f"Using {n_components} latent components")

svd = TruncatedSVD(n_components=n_components, random_state=42)
user_factors = svd.fit_transform(train_csr)
item_factors = svd.components_.T

# normalize for cosine comparisons
user_factors = normalize(user_factors, axis=1)
item_factors = normalize(item_factors, axis=1)

print("✅ user_factors:", user_factors.shape)
print("✅ item_factors:", item_factors.shape)


Using 50 latent components
✅ user_factors: (800, 50)
✅ item_factors: (13873, 50)


In [18]:
# ============================================================
# 🧠 Hybrid Recommendation System
# ============================================================

import numpy as np

# mapping index to product_id
index_to_item_list = [index_to_item[i] for i in range(len(index_to_item))]
prodid_to_idx = {pid: i for i, pid in enumerate(product_meta['product_id'].values)}

def recommend_for_user(user_id, top_k=10, alpha=0.6):
    """
    alpha → weight for collaborative filtering (0–1)
    0 = pure content-based, 1 = pure collaborative
    """
    if user_id not in user_to_index:
        return []
    uidx = user_to_index[user_id]
    coll_scores = item_factors.dot(user_factors[uidx])
    
    # Compute content-based similarity (mean of items the user interacted with)
    user_items = interactions[interactions['user_id'] == user_id]['product_id'].unique().tolist()
    content_scores = np.zeros(len(item_ids))
    for pid in user_items:
        if pid in prodid_to_idx:
            idx = prodid_to_idx[pid]
            sim = prod_cosine[idx]
            content_scores += sim
    if len(user_items) > 0:
        content_scores /= len(user_items)

    # Normalize and combine
    coll_norm = (coll_scores - coll_scores.min()) / (coll_scores.max() - coll_scores.min() + 1e-9)
    cont_norm = (content_scores - content_scores.min()) / (content_scores.max() - content_scores.min() + 1e-9)
    hybrid = alpha * coll_norm + (1 - alpha) * cont_norm

    # Exclude items already seen
    seen = set(user_items)
    candidates = [(index_to_item_list[i], hybrid[i]) for i in range(len(hybrid)) if index_to_item_list[i] not in seen]
    candidates.sort(key=lambda x: x[1], reverse=True)

    return [pid for pid, score in candidates[:top_k]]

# Try one random user
sample_user = np.random.choice(interactions['user_id'].unique())
recs = recommend_for_user(sample_user, top_k=10, alpha=0.6)
print(f"Example user: {sample_user}")
print("Top Recommendations:", recs[:10])


Example user: U102
Top Recommendations: ['https://www.amazon.in/Paninaro-Afghani-Oxidised-Silver-Necklace/dp/B09746PB5S/ref=sr_1_18103?qid=1679160577&s=jewelry&sr=1-18103', 'https://www.amazon.in/Liberty-Fortune-VCL-3-Formal-Shoes/dp/B0BVZ4Y5RB/ref=sr_1_8658?qid=1679148215&s=shoes&sr=1-8658', 'https://www.amazon.in/SS-Art-Land-Decorations-Activities/dp/B0BHPCRHNN/ref=sr_1_7960?qid=1679215230&s=beauty&sr=1-7960', 'https://www.amazon.in/INDO-ERA-Straight-Trouser-Blue_RRRRR1996_Medium/dp/B097KCSH1S/ref=sr_1_7187?qid=1679152999&s=apparel&sr=1-7187', 'https://www.amazon.in/Peter-England-Cotton-Classic-PETRMRGB249085_Navy_XL/dp/B0B575L4ZY/ref=sr_1_295?qid=1679142483&s=apparel&sr=1-295', 'https://www.amazon.in/Sukkhi-Classic-Plated-Necklace-CBMIX104228/dp/B09Y3D2QWC/ref=sr_1_13006?qid=1679160394&s=jewelry&sr=1-13006', 'https://www.amazon.in/PRO-GYM-Compression-T-Shirt-Athletic/dp/B08VW4T7DP/ref=sr_1_3264?qid=1679217401&s=sports&sr=1-3264', 'https://www.amazon.in/Lining-Airforce-Badminton-Racq

In [19]:
# ============================================================
# 📊 Evaluate model using Precision@K and Recall@K
# ============================================================

def precision_at_k(recs, gt, k):
    if not recs or not gt:
        return 0
    return len(set(recs[:k]) & set(gt)) / k

def recall_at_k(recs, gt, k):
    if not recs or not gt:
        return 0
    return len(set(recs[:k]) & set(gt)) / len(gt)

# Split small test set: hold out one item per user
test_df = interactions.groupby('user_id').head(1)
train_df = interactions[~interactions.index.isin(test_df.index)]

def evaluate(test_df, k=10, alpha=0.6):
    users = test_df['user_id'].unique()
    precisions, recalls = [], []
    for u in users:
        gt = test_df[test_df['user_id'] == u]['product_id'].tolist()
        recs = recommend_for_user(u, top_k=k, alpha=alpha)
        precisions.append(precision_at_k(recs, gt, k))
        recalls.append(recall_at_k(recs, gt, k))
    return np.mean(precisions), np.mean(recalls)

for alpha in [0.0, 0.3, 0.5, 0.7, 1.0]:
    p, r = evaluate(test_df, k=10, alpha=alpha)
    print(f"α={alpha:.1f} → Precision@10={p:.4f}, Recall@10={r:.4f}")


α=0.0 → Precision@10=0.0000, Recall@10=0.0000
α=0.3 → Precision@10=0.0000, Recall@10=0.0000
α=0.5 → Precision@10=0.0000, Recall@10=0.0000
α=0.7 → Precision@10=0.0000, Recall@10=0.0000
α=1.0 → Precision@10=0.0000, Recall@10=0.0000


In [20]:
# ============================================================
# 💾 Save model artifacts
# ============================================================
import joblib

joblib.dump(tfidf, "/kaggle/working/tfidf_vectorizer.joblib")
joblib.dump(product_meta, "/kaggle/working/product_meta.joblib")
joblib.dump({
    "user_factors": user_factors,
    "item_factors": item_factors,
    "user_to_index": user_to_index,
    "index_to_item": index_to_item
}, "/kaggle/working/svd_factors.joblib")

print("✅ Artifacts saved in /kaggle/working/")
print("Files ready for Streamlit deployment:")
!ls -lh /kaggle/working/


✅ Artifacts saved in /kaggle/working/
Files ready for Streamlit deployment:
total 769M
-rw-r--r-- 1 root root 701M Nov  5 05:00 annoy_index.ann
-rw-r--r-- 1 root root 6.2M Nov  5 05:06 product_meta.joblib
-rw-r--r-- 1 root root  50M Nov  5 05:00 similarity_map.joblib
-rw-r--r-- 1 root root 7.4M Nov  5 05:06 svd_factors.joblib
-rw-r--r-- 1 root root 4.9M Nov  5 05:00 svd_model.joblib
-rw-r--r-- 1 root root 176K Nov  5 05:06 tfidf_vectorizer.joblib


In [21]:
for u in random.sample(interactions['user_id'].unique().tolist(), 3):
    recs = recommend_for_user(u, top_k=5, alpha=0.6)
    print(f"\nUser {u} → Recommendations:")
    print(product_meta[product_meta['product_id'].isin(recs)][['title','category']].head())



User U166 → Recommendations:
                                                   title  \
296    GM 3012 3 Pin Travel Universal Multi-Plug with...   
999    Lee Cooper Unisex Polyester Super Breathable a...   
7562                 VMart Men Cotton Medium Waist Jeans   
10891                              Allen Solly Men Shirt   
11192  Shopnet Wireless Bluetooth Speaker TG113 For M...   

                                  category  
296    tv, audio & cameras All Electronics  
999             accessories Bags & Luggage  
7562                  men's clothing Jeans  
10891                men's clothing Shirts  
11192         tv, audio & cameras Speakers  

User U149 → Recommendations:
                                                   title  \
5187   Avsar 18k (750) Yellow Gold and Diamond Pendan...   
6641   Home9ine Matte Poly Duck Red Color Digital Pri...   
10260  CuTech New 1080p HD Bulb Light Camera Fish Eye...   
10676                             Jack & Jones Men Shirt   
13441  St

In [22]:
def train_test_overlap(df, user_col='user_id', item_col='product_id'):
    """Guarantee that held-out items exist in training set."""
    train_df, test_df = [], []
    for u, group in df.groupby(user_col):
        if len(group) <= 2:
            train_df.append(group)
        else:
            test_df.append(group.sample(1))
            train_df.append(group.drop(test_df[-1].index))
    train_df = pd.concat(train_df)
    test_df = pd.concat(test_df)
    return train_df, test_df

train_df, test_df = train_test_overlap(interactions)


In [23]:
# View some available synthetic users
list(interactions['user_id'].unique())[:20]


['U0',
 'U1',
 'U2',
 'U3',
 'U4',
 'U5',
 'U6',
 'U7',
 'U8',
 'U9',
 'U10',
 'U11',
 'U12',
 'U13',
 'U14',
 'U15',
 'U16',
 'U17',
 'U18',
 'U19']

In [24]:
product_meta[['product_id', 'title']].head(10)


,product_id,title
0,https://www.amazon.in/Voltas-Inverter-Split-Co...,"Voltas 1.5 Ton, 5 Star, Inverter Split AC(Copp..."
1,https://www.amazon.in/Blue-Star-Portable-Coppe...,Blue Star 1 Ton Fixed Speed Portable AC (Coppe...
2,https://www.amazon.in/Panasonic-Convertible-ad...,Panasonic 2 Ton 5 Star Wi-Fi Inverter Smart Sp...
3,https://www.amazon.in/Daikin-Inverter-Copper-F...,Daikin 1.5 Ton 5 Star Inverter Split AC (Coppe...
4,https://www.amazon.in/Hisense-Inverter-Copper-...,Hisense 1.0 Ton 5 Star Inverter Split AC (Copp...
5,https://www.amazon.in/Voltas-Split-Hot-Cold-Wh...,"Voltas 1.5 Ton Antibacterial Coating, Heater, ..."
6,https://www.amazon.in/Daikin-Inverter-Copper-F...,Daikin 1.8 Ton 5 Star Inverter Split AC (Coppe...
7,https://www.amazon.in/Voltas-Inverter-Copper-1...,Voltas 1 Ton 3 Star Inverter Split AC (Copper ...
8,https://www.amazon.in/Voltas-Adjustable-Invert...,Voltas 1.5 Ton 3 Star Adjustable Inverter Copp...
9,https://www.amazon.in/Voltas-Split-Copper-183D...,Voltas 1.5 Ton 3 Star Non-Inverter Split AC (1...


In [26]:
# Rebuild a small Annoy index (<= ~25MB) from existing artifacts or product_meta
import os, joblib, time, gzip, shutil
import numpy as np, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

start = time.time()
print("Start: rebuild compact Annoy index")

# Parameters (tune if needed)
TARGET_DIM = 32        # final embedding dimension (32 usually enough)
N_TREES = 10           # number of Annoy trees (smaller -> smaller file, less accuracy)
ANN_FILENAME = "annoy_index_small.ann"
ANN_GZ = ANN_FILENAME + ".gz"
SVD_MODEL_SMALL = "svd_model_small.joblib"
PRODUCT_META_SMALL = "product_meta_small.joblib"
TFIDF_NAME = "tfidf_vectorizer.joblib"

# 1) Load product_meta (try joblib first, then look for a DataFrame in memory)
product_meta = None
if os.path.exists("product_meta.joblib"):
    try:
        product_meta = joblib.load("product_meta.joblib")
        print("Loaded product_meta.joblib")
    except Exception as e:
        print("Failed to load product_meta.joblib:", e)
if product_meta is None and 'product_meta' in globals():
    product_meta = globals()['product_meta']
    print("Using product_meta from notebook globals")

if product_meta is None:
    raise RuntimeError("product_meta not found. Place product_meta.joblib in working dir or define product_meta in the notebook.")

# 2) Ensure there is a text column (prefer text_clean > text > title > name)
text_col = None
for cand in ['text_clean','text','title','name','product_name']:
    if cand in product_meta.columns:
        text_col = cand
        break

# If no candidate, try to build 'text' by joining obvious columns
if text_col is None:
    print("No text-like column found. Building 'text' from object columns...")
    # choose up to 3 object columns that are not likely urls/images
    obj_cols = [c for c in product_meta.columns if product_meta[c].dtype == 'object']
    def is_url_col(col):
        sample = product_meta[col].dropna().astype(str).head(6).tolist()
        if not sample: return False
        return sum(1 for s in sample if any(x in s.lower() for x in ('http','www.','.jpg','.png','/dp/','amazon'))) >= 1
    obj_cols = [c for c in obj_cols if not is_url_col(c)]
    if not obj_cols:
        # fallback: use product_id as token
        if 'product_id' in product_meta.columns:
            product_meta['text'] = product_meta['product_id'].astype(str)
            text_col = 'text'
        else:
            raise RuntimeError("No text columns available and no product_id to fallback to.")
    else:
        chosen = obj_cols[:3]
        print("Using columns to build text:", chosen)
        def build_text(row):
            parts = []
            for c in chosen:
                v = row.get(c)
                if pd.isna(v): continue
                s = str(v).strip()
                if s and not s.lower().startswith("http"):
                    parts.append(s)
            return " ".join(parts)
        product_meta['text'] = product_meta.apply(build_text, axis=1)
        text_col = 'text'

print("Using text column:", text_col)
# create text_clean if missing
if 'text_clean' not in product_meta.columns:
    import re
    def clean_text(s):
        if pd.isna(s): return ""
        s = str(s).lower()
        s = re.sub(r'[^a-z0-9\s]', ' ', s)
        s = re.sub(r'\s+', ' ', s).strip()
        return s
    product_meta['text_clean'] = product_meta[text_col].fillna("").map(clean_text)

n_items = len(product_meta)
if n_items == 0:
    raise RuntimeError("product_meta is empty (0 rows). Nothing to index.")

print(f"Products to index: {n_items}; sample cleaned text:")
print(product_meta['text_clean'].astype(str).head(3).tolist())

# 3) Load TF-IDF if available, else fit a small one on current product_meta
tfidf = None
if os.path.exists(TFIDF_NAME):
    try:
        tfidf = joblib.load(TFIDF_NAME)
        print("Loaded existing TF-IDF vectorizer:", TFIDF_NAME)
    except Exception as e:
        print("Failed to load TF-IDF from joblib:", e)
if tfidf is None:
    print("Fitting new TF-IDF vectorizer (max_features=5000)...")
    tfidf = TfidfVectorizer(max_features=5000, stop_words='english', token_pattern=r'(?u)\b[A-Za-z0-9_]{2,}\b', min_df=1)
    tfidf.fit(product_meta['text_clean'].astype(str).values)
    joblib.dump(tfidf, TFIDF_NAME)
    print("Saved new TF-IDF to", TFIDF_NAME)

# 4) Transform to sparse TF-IDF
print("Transforming text to TF-IDF (sparse)...")
tfidf_matrix = tfidf.transform(product_meta['text_clean'].astype(str).values)
print("TF-IDF shape:", tfidf_matrix.shape)

# 5) Compute compact SVD embeddings
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

n_comp = TARGET_DIM
print(f"Computing TruncatedSVD with n_components={n_comp} ... (dense embedding)")
svd_small = TruncatedSVD(n_components=n_comp, random_state=42)
item_embeds_small = svd_small.fit_transform(tfidf_matrix)  # (n_items, n_comp)
item_embeds_small = normalize(item_embeds_small, axis=1)
print("Embeddings shape:", item_embeds_small.shape)

# 6) Build compact Annoy index
try:
    from annoy import AnnoyIndex
except Exception:
    raise RuntimeError("annoy not installed. Run `pip install annoy` in your environment.")

dim = item_embeds_small.shape[1]
ann = AnnoyIndex(dim, metric='angular')
print(f"Adding {n_items} vectors to Annoy index (dim={dim}) ...")
for i in range(n_items):
    ann.add_item(i, item_embeds_small[i].astype('float32'))

print(f"Building Annoy with {N_TREES} trees (this may take a bit)...")
ann.build(N_TREES)
ann.save(ANN_FILENAME)
print("Saved", ANN_FILENAME)

# 7) gzip-compress the index for GitHub (reduces size)
print("Gzipping", ANN_FILENAME, "->", ANN_GZ)
with open(ANN_FILENAME, 'rb') as f_in, gzip.open(ANN_GZ, 'wb', compresslevel=9) as f_out:
    shutil.copyfileobj(f_in, f_out)
gz_size = os.path.getsize(ANN_GZ) / (1024*1024)
orig_size = os.path.getsize(ANN_FILENAME) / (1024*1024)
print(f"Original ANN size: {orig_size:.2f} MB, gzipped: {gz_size:.2f} MB")

# 8) Save small SVD & small product_meta (to use in deployment)
joblib.dump({'svd': svd_small}, SVD_MODEL_SMALL)
product_meta[['product_id','title','text_clean']].to_pickle(PRODUCT_META_SMALL)
print("Saved compact artifacts:", SVD_MODEL_SMALL, PRODUCT_META_SMALL)

elapsed = time.time() - start
print(f"Done in {elapsed:.1f}s")


Start: rebuild compact Annoy index
Loaded product_meta.joblib
Using text column: text
Products to index: 13873; sample cleaned text:
['voltas 1 5 ton 5 star inverter split ac copper 4 in 1 adjustable mode anti dust filter 2023 model 185v dazj white voltas 1 5 ton 5 star inverter split ac copper 4 in 1 adjustable mode anti dust filter 2023 model 185v dazj white appliances air conditioners', 'blue star 1 ton fixed speed portable ac copper anti bacterial silver coating self diagnosis comfort sleep modes auto c blue star 1 ton fixed speed portable ac copper anti bacterial silver coating self diagnosis comfort sleep modes auto c appliances air conditioners', 'panasonic 2 ton 5 star wi fi inverter smart split ac copper 7 in 1 convertible with additional ai mode twin cool pm 0 panasonic 2 ton 5 star wi fi inverter smart split ac copper 7 in 1 convertible with additional ai mode twin cool pm 0 appliances air conditioners']
Loaded existing TF-IDF vectorizer: tfidf_vectorizer.joblib
Transforming

In [27]:
# Compact similarity map generator for deployment (GitHub/Streamlit friendly)
import joblib, os
import numpy as np
from annoy import AnnoyIndex

# --- Parameters ---
ANN_PATH = "annoy_index_small.ann"
OUT_PATH = "similarity_map_small.joblib"
N_NEIGHBORS = 5  # reduce to top 5–10 neighbors to shrink file

# --- Load product_meta ---
if os.path.exists("product_meta_small.joblib"):
    try:
        product_meta = joblib.load("product_meta_small.joblib")
        if isinstance(product_meta, pd.DataFrame):
            product_meta = product_meta
        else:
            product_meta = pd.DataFrame(product_meta)
    except Exception:
        product_meta = pd.read_pickle("product_meta_small.joblib")
elif os.path.exists("product_meta.joblib"):
    product_meta = joblib.load("product_meta.joblib")
else:
    raise FileNotFoundError("product_meta_small.joblib or product_meta.joblib not found!")

# --- Load Annoy index ---
dim = 32  # same as you used when building small Annoy index
ann = AnnoyIndex(dim, metric="angular")
ann.load(ANN_PATH)
print(f"✅ Loaded Annoy index ({dim} dims)")

# --- Build smaller similarity map ---
product_ids = product_meta['product_id'].astype(str).tolist()
n_items = len(product_ids)
small_map = {}

print(f"Building compact similarity map (top {N_NEIGHBORS} per item, {n_items} items)...")
for i, pid in enumerate(product_ids):
    nn = ann.get_nns_by_item(i, N_NEIGHBORS + 1)
    # remove self
    nn = [j for j in nn if j != i][:N_NEIGHBORS]
    small_map[pid] = [product_ids[j] for j in nn]
    if i % 5000 == 0 and i > 0:
        print(f"Processed {i}/{n_items} items")

# --- Save compressed ---
joblib.dump(small_map, OUT_PATH, compress=('xz', 3))  # strong compression, small size
size_mb = os.path.getsize(OUT_PATH) / (1024 * 1024)
print(f"✅ Saved {OUT_PATH} ({size_mb:.2f} MB)")


✅ Loaded Annoy index (32 dims)
Building compact similarity map (top 5 per item, 13873 items)...
Processed 5000/13873 items
Processed 10000/13873 items
✅ Saved similarity_map_small.joblib (0.61 MB)
